# Solar Panel Economics Calculator

This notebook demonstrates how to use the solar panel economics calculator to estimate the financial performance of a solar PV system.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from solar_production import calculate_solar_economics

## Load Data

First, we load the required input data: electricity consumption, spot prices, and solar production.

In [ ]:
# Load consumption data
consumption_df = pd.read_csv('consumption.csv', parse_dates=['START_TIME'])
consumption_df = consumption_df.set_index('START_TIME')
consumption_hourly = consumption_df['QUANTITY_KWH']

# Display first few rows
consumption_hourly.head()

In [ ]:
# Load spot prices
spot_prices_df = pd.read_excel('spotpriser.xlsx', parse_dates=True, index_col=0)
spot_price_hourly = spot_prices_df.iloc[:, 0]  # Assuming the first column contains prices

# Display first few rows
spot_price_hourly.head()

## Create Sample Solar Production Data

For this demo, we'll create a simulated solar production curve. In a real scenario, you would load actual production data or use a simulation tool like PVGIS.

In [ ]:
# Create a simplified solar production model for demonstration
# In real applications, use actual data or proper simulation tools

def simulate_solar_production(start_date, days=365, peak_capacity=10.0):
    """Create a simple simulated solar production dataset"""
    date_range = pd.date_range(start=start_date, periods=24*days, freq='H')
    
    # Day of year to approximate seasonal variations
    day_of_year = date_range.dayofyear
    
    # Hour of day to model daily production curve
    hour_of_day = date_range.hour
    
    # Seasonal factor: more production in summer
    seasonal_factor = np.sin((day_of_year - 172) * 2 * np.pi / 365) * 0.5 + 0.5
    
    # Daily factor: production follows a bell curve during daylight hours
    daily_factor = np.exp(-((hour_of_day - 12)**2) / 20)
    
    # Combine factors and scale by peak capacity
    production = peak_capacity * seasonal_factor * daily_factor
    
    # Add some random variation
    noise = np.random.normal(0, 0.1, len(production))
    production = production + noise * production
    
    # Ensure non-negative values
    production = np.maximum(production, 0)
    
    return pd.Series(production, index=date_range)

# Get the start date from consumption data
start_date = consumption_hourly.index[0]

# Simulate solar production for 10 kWp system
production_hourly = simulate_solar_production(start_date, peak_capacity=10.0)

# Plot sample daily production
plt.figure(figsize=(10, 6))
production_hourly['2022-08-01':'2022-08-07'].plot()
plt.title('Simulated Solar Production (First Week)')
plt.ylabel('Power (kW)')
plt.grid(True)
plt.show()

## Calculate Solar System Economics

Now we'll use the `calculate_solar_economics` function to evaluate the financial performance of the solar system.

In [ ]:
# Make sure all series have the same time range
common_index = consumption_hourly.index.intersection(spot_price_hourly.index).intersection(production_hourly.index)
consumption = consumption_hourly.loc[common_index]
spot_prices = spot_price_hourly.loc[common_index]
production = production_hourly.loc[common_index]

# Set parameters
investment_cost = 150000  # NOK for a 10 kWp system
lifetime_years = 25
degradation_per_year = 0.005  # 0.5% degradation per year
grid_tariff = 0.5  # NOK/kWh
system_losses = 0.14  # System losses (inverter efficiency, wiring, etc.)

# Calculate economics
results_df = calculate_solar_economics(
    production_hourly=production,
    spot_price_hourly=spot_prices,
    consumption_hourly=consumption,
    investment_cost=investment_cost,
    lifetime_years=lifetime_years,
    degradation_per_year=degradation_per_year,
    grid_tariff=grid_tariff,
    system_losses=system_losses,
    inflation=0.02
)

# Display results
results_df

In [ ]:
# Plot cumulative cashflow
plt.figure(figsize=(12, 6))
plt.plot(results_df['Year'], results_df['Cumulative (NOK)'], marker='o')
plt.axhline(y=investment_cost, color='r', linestyle='-', label='Investment Cost')
plt.title('Cumulative Cashflow')
plt.xlabel('Year')
plt.ylabel('NOK')
plt.grid(True)
plt.legend()
plt.show()

## Sensitivity Analysis

Let's explore how different parameters affect the economics.

In [ ]:
# Function to calculate payback period
def calculate_payback(capacity_kw, investment_per_kw=15000, grid_tariff=0.5):
    investment = capacity_kw * investment_per_kw
    
    # Scale production by capacity
    scaled_production = production * (capacity_kw / 10.0)
    
    results = calculate_solar_economics(
        production_hourly=scaled_production,
        spot_price_hourly=spot_prices,
        consumption_hourly=consumption,
        investment_cost=investment,
        lifetime_years=lifetime_years,
        degradation_per_year=degradation_per_year,
        grid_tariff=grid_tariff,
        system_losses=system_losses
    )
    
    # Find payback year
    for index, row in results.iterrows():
        if row['Cumulative (NOK)'] >= investment:
            return row['Year']
    
    return float('inf')  # No payback within lifetime

# Test different system sizes
capacities = np.arange(5, 21, 2.5)
payback_years = [calculate_payback(cap) for cap in capacities]

plt.figure(figsize=(10, 6))
plt.plot(capacities, payback_years, marker='o')
plt.title('Payback Period vs. System Size')
plt.xlabel('System Size (kWp)')
plt.ylabel('Payback Period (Years)')
plt.grid(True)
plt.show()

## Conclusion

This notebook demonstrates how to use the solar economics calculator to analyze the financial performance of a solar PV system. The analysis includes:

- Loading and preparing input data
- Calculating financial metrics over the system lifetime
- Visualizing cumulative cashflow and payback period
- Performing sensitivity analysis on system size

You can extend this analysis by:
- Exploring different system configurations
- Adding more sophisticated production modeling
- Analyzing the impact of electricity price scenarios
- Including battery storage in the model